# Evaluating Home Field Advantage During the Regular Season

## Import needed packages and config

In [1]:
import requests
import config

## Make API Call to grab 2018 reg season data

In [2]:
api_key = config.nhl_api_key
url = config.nhl_url

headers = {'Ocp-Apim-Subscription-Key' : '{}'.format(api_key)}

data = []
response = requests.get(url, headers=headers)
data_ = response.json()
data.append(data_)

In [3]:
data = data[0] # data came back as list of lists of dictionaries.

In [5]:
parsed_data = []
for game in data:
    parsed_game = {}
    parsed_game['game_id'] = game['GameID']
    parsed_game['month'] = game['DateTime'][5:7]
    parsed_game['away_team'] = game['AwayTeam']
    parsed_game['away_team_score'] = game['AwayTeamScore']
    parsed_game['away_team_moneyline'] = game['PointSpreadAwayTeamMoneyLine']
    parsed_game['home_team'] = game['HomeTeam']
    parsed_game['home_team_score'] = game['HomeTeamScore']
    parsed_game['home_team_moneyline'] = game['PointSpreadHomeTeamMoneyLine']
    parsed_game['spread'] = game['PointSpread']
    parsed_data.append(parsed_game)

In [6]:
#define db_name and create_query

table_name = 'nhl'

nhl_create_query = """ 
CREATE TABLE {} (
     game_id varchar(11) NOT NULL primary key,
    month varchar(2) not null,
    away_team varchar(11) not null,
    away_team_score int(3) not null,
    home_team varchar(11) not null,
    home_team_score int(3) not null,
    away_team_moneyline int(4) not null,
    home_team_moneyline int(4) not null,
    spread float(5) not null
    )
 """.format(table_name)

In [7]:
#get data into an insertable format

nhl_data_to_insert = []

for game in parsed_data:
    game_to_insert = (
        game['game_id'], 
        game['month'], 
        game['away_team'],
        game['away_team_score'],
        game['home_team'], 
        game['home_team_score'],
        game['away_team_moneyline'],
        game['home_team_moneyline'],
        game['spread'] 
    )
    nhl_data_to_insert.append(game_to_insert)

In [11]:
#define insert_query

table_name = 'nhl'

nhl_insert_query = """
INSERT INTO {} (
    game_id,
    month,
    away_team,
    away_team_score,
    home_team,
    home_team_score,
    away_team_moneyline,
    home_team_moneyline,
    spread
    )
VALUES (
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s
    )
""".format(table_name)